# 🛠️ Avanceret Værktøjsbrug med GitHub-modeller (.NET)

## 📋 Læringsmål

Denne notebook demonstrerer mønstre for integration af værktøjer på virksomhedsniveau ved brug af Microsoft Agent Framework i .NET med GitHub-modeller. Du vil lære at bygge avancerede agenter med flere specialiserede værktøjer, der udnytter C#'s stærke typning og .NET's funktioner til erhvervsbrug.

**Avancerede værktøjsfunktioner, du vil mestre:**
- 🔧 **Multi-værktøjsarkitektur**: Opbygning af agenter med flere specialiserede kapaciteter
- 🎯 **Type-sikker værktøjsudførelse**: Udnyttelse af C#'s validering ved kompilering
- 📊 **Virksomhedsværktøjsmønstre**: Produktionsklar værktøjsdesign og fejlhåndtering
- 🔗 **Værktøjssammensætning**: Kombinering af værktøjer til komplekse forretningsarbejdsgange

## 🎯 Fordele ved .NET-værktøjsarkitektur

### Funktioner til erhvervsbrug
- **Validering ved kompilering**: Stærk typning sikrer korrekthed af værktøjsparametre
- **Dependency Injection**: IoC-containerintegration til værktøjsstyring
- **Async/Await-mønstre**: Ikke-blokerende værktøjsudførelse med korrekt ressourcehåndtering
- **Struktureret logning**: Indbygget logningsintegration til overvågning af værktøjsudførelse

### Produktionsklare mønstre
- **Fejlhåndtering**: Omfattende fejlhåndtering med typede undtagelser
- **Ressourcehåndtering**: Korrekte dispose-mønstre og hukommelseshåndtering
- **Performanceovervågning**: Indbyggede metrikker og performance-tællere
- **Konfigurationsstyring**: Type-sikker konfiguration med validering

## 🔧 Teknisk Arkitektur

### Centrale .NET-værktøjskomponenter
- **Microsoft.Extensions.AI**: Enhedligt abstraktionslag for værktøjer
- **Microsoft.Agents.AI**: Værktøjsorkestrering på virksomhedsniveau
- **GitHub Models Integration**: Højtydende API-klient med forbindelsespooling

### Værktøjsudførelsespipeline
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Værktøjskategorier & Mønstre

### 1. **Databehandlingsværktøjer**
- **Inputvalidering**: Stærk typning med dataannoteringer
- **Transformationsoperationer**: Type-sikker datakonvertering og formatering
- **Forretningslogik**: Domænespecifikke beregnings- og analysetools
- **Outputformatering**: Generering af strukturerede svar

### 2. **Integrationsværktøjer** 
- **API-forbindelser**: RESTful serviceintegration med HttpClient
- **Databaseværktøjer**: Entity Framework-integration til dataadgang
- **Filoperationer**: Sikker filsystemhåndtering med validering
- **Eksterne tjenester**: Mønstre for integration af tredjepartstjenester

### 3. **Hjælpeværktøjer**
- **Tekstbehandling**: Strengmanipulation og formateringsværktøjer
- **Dato/tid-operationer**: Kulturbevidste beregninger af dato/tid
- **Matematiske værktøjer**: Præcisionsberegninger og statistiske operationer
- **Valideringsværktøjer**: Validering af forretningsregler og dataverifikation

## ⚙️ Forudsætninger & Opsætning

**Udviklingsmiljø:**
- .NET 9.0 SDK eller nyere
- Visual Studio 2022 eller VS Code med C#-udvidelse
- Adgang til GitHub Models API

**Påkrævede NuGet-pakker:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Miljøkonfiguration (.env-fil):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Klar til at bygge agenter på virksomhedsniveau med kraftfulde, type-sikre værktøjsfunktioner i .NET? Lad os designe nogle professionelle løsninger! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal det bemærkes, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
